In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
#import missingno as msno

# matplotlib
from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

# scipy specifics
from scipy import stats as sps
from scipy.interpolate import interp1d

In [2]:
try:
    from urllib.request import Request, urlopen  # Python 3
except ImportError:
    from urllib2 import Request, urlopen  # Python 2

req = Request('https://cloud.minsa.gob.pe/s/AC2adyLkHCKjmfm/download')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)

test = pd.read_csv(content , sep=";", na_values='EN INVESTIGACIÓN')
print(test)

         FECHA_CORTE                                  UUID DEPARTAMENTO  \
0           20210817  d4cdd94c-3268-4e80-8296-d343cec109fb   LAMBAYEQUE   
1           20210817  586bce7f-3007-4e00-80a3-47fb44fa05ef         LIMA   
2           20210817  0520eb2d-b20b-4efb-b391-b155c24e2501         LIMA   
3           20210817  c59cd628-7031-48be-af21-9dd22d403ff9         LIMA   
4           20210817  900bbb90-10f2-4c0f-be7e-e8b34b0523cd         LIMA   
...              ...                                   ...          ...   
2137290     20210817  a84f4445-9608-4851-84d2-59b0b159cf6e         LIMA   
2137291     20210817  92c03f05-13f7-411b-8f8f-07932efdea6a         LIMA   
2137292     20210817  a589547e-9ba5-404f-8a47-cae3bda70f13         LIMA   
2137293     20210817  d7ce8e32-ec0d-4f41-a9e1-30b536006556         LIMA   
2137294     20210817  339cae67-05d9-48f1-a5d1-7d993d790fe7         LIMA   

         PROVINCIA                DISTRITO METODODX  EDAD       SEXO  \
0        FERREÑAFE         

In [3]:
test['FECHA_RESULTADO'] = pd.to_datetime(test['FECHA_RESULTADO'].astype(str), format='%Y%m%d')
test.tail()

,FECHA_CORTE,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO
2137290,20210817,a84f4445-9608-4851-84d2-59b0b159cf6e,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PCR,64.0,MASCULINO,2021-01-07,150132.0
2137291,20210817,92c03f05-13f7-411b-8f8f-07932efdea6a,LIMA,LIMA,LIMA,PCR,62.0,MASCULINO,2020-12-25,150101.0
2137292,20210817,a589547e-9ba5-404f-8a47-cae3bda70f13,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PCR,61.0,MASCULINO,2020-12-10,150132.0
2137293,20210817,d7ce8e32-ec0d-4f41-a9e1-30b536006556,LIMA,LIMA,SAN MIGUEL,PCR,62.0,MASCULINO,2020-12-12,150136.0
2137294,20210817,339cae67-05d9-48f1-a5d1-7d993d790fe7,LIMA,LIMA,SAN ISIDRO,PCR,60.0,MASCULINO,2020-12-15,150131.0


In [4]:
test.isnull().sum()

FECHA_CORTE             0
UUID                    0
DEPARTAMENTO            0
PROVINCIA          107538
DISTRITO           107538
METODODX                0
EDAD                  347
SEXO                    1
FECHA_RESULTADO      2023
UBIGEO             107538
dtype: int64

In [5]:
indice_departamento = pd.read_csv('https://raw.githubusercontent.com/annaabsi/git-scraper-covid19/main/resultados/positivos_por_departamentos.csv')
indice_departamento

,DEPARTAMENTO,METODODX,POBLACION,INDICE
0,AMAZONAS,31042,426806,7273
1,ANCASH,75502,1180638,6395
2,APURIMAC,25720,430736,5971
3,AREQUIPA,112546,1497438,7516
4,AYACUCHO,32556,668213,4872
5,CAJAMARCA,64567,1453711,4442
6,CALLAO,99074,1129854,8769
7,CUSCO,71531,1357075,5271
8,HUANCAVELICA,15839,365317,4336
9,HUANUCO,35013,760267,4605


In [6]:
test = pd.merge(test, indice_departamento,
                        how="left", on=["DEPARTAMENTO"])
test

,FECHA_CORTE,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,METODODX_y,POBLACION,INDICE
0,20210817,d4cdd94c-3268-4e80-8296-d343cec109fb,LAMBAYEQUE,FERREÑAFE,FERREÑAFE,PR,35.0,FEMENINO,2020-03-29,140201.0,60386,1310785,4607
1,20210817,586bce7f-3007-4e00-80a3-47fb44fa05ef,LIMA,LIMA,CHORRILLOS,PR,36.0,MASCULINO,2020-03-30,150108.0,932895,10628470,8777
2,20210817,0520eb2d-b20b-4efb-b391-b155c24e2501,LIMA,LIMA,LIMA,PR,1.0,FEMENINO,2020-03-30,150101.0,932895,10628470,8777
3,20210817,c59cd628-7031-48be-af21-9dd22d403ff9,LIMA,LIMA,LIMA,PR,65.0,FEMENINO,2020-03-30,150101.0,932895,10628470,8777
4,20210817,900bbb90-10f2-4c0f-be7e-e8b34b0523cd,LIMA,LIMA,LIMA,PR,32.0,FEMENINO,2020-03-30,150101.0,932895,10628470,8777
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137290,20210817,a84f4445-9608-4851-84d2-59b0b159cf6e,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PCR,64.0,MASCULINO,2021-01-07,150132.0,932895,10628470,8777
2137291,20210817,92c03f05-13f7-411b-8f8f-07932efdea6a,LIMA,LIMA,LIMA,PCR,62.0,MASCULINO,2020-12-25,150101.0,932895,10628470,8777
2137292,20210817,a589547e-9ba5-404f-8a47-cae3bda70f13,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PCR,61.0,MASCULINO,2020-12-10,150132.0,932895,10628470,8777
2137293,20210817,d7ce8e32-ec0d-4f41-a9e1-30b536006556,LIMA,LIMA,SAN MIGUEL,PCR,62.0,MASCULINO,2020-12-12,150136.0,932895,10628470,8777


In [7]:
poblacion_csv = pd.read_csv('poblacion_provincia.csv')
poblacion_csv

,UBIGEO,PROVINCIA,POBLACION
0,10100,CHACHAPOYAS,63188
1,10200,BAGUA,84672
2,10300,BONGARA,26830
3,10400,CONDORCANQUI,51344
4,10500,LUYA,47827
...,...,...,...
191,240300,ZARUMILLA,56038
192,250100,CORONEL PORTILLO,447733
193,250200,ATALAYA,61049
194,250300,PADRE ABAD,77044


In [8]:
poblacion_dict = poblacion_csv.to_dict('split')
poblacion_dict['data']

[[10100, 'CHACHAPOYAS', 63188],
 [10200, 'BAGUA', 84672],
 [10300, 'BONGARA', 26830],
 [10400, 'CONDORCANQUI', 51344],
 [10500, 'LUYA', 47827],
 [10600, 'RODRIGUEZ DE MENDOZA', 33651],
 [10700, 'UTCUBAMBA', 119294],
 [20100, 'HUARAZ', 185276],
 [20200, 'AIJA', 6433],
 [20300, 'ANTONIO RAYMONDI', 13950],
 [20400, 'ASUNCION', 7710],
 [20500, 'BOLOGNESI', 24012],
 [20600, 'CARHUAZ', 50007],
 [20700, 'CARLOS FERMIN FITZCARRALD', 18496],
 [20800, 'CASMA', 57256],
 [20900, 'CORONGO', 8017],
 [21000, 'HUARI', 63264],
 [21100, 'HUARMEY', 33066],
 [21200, 'HUAYLAS', 56557],
 [21300, 'MARISCAL LUZURIAGA', 21787],
 [21400, 'OCROS', 7224],
 [21500, 'PALLASCA', 24371],
 [21600, 'POMABAMBA', 26675],
 [21700, 'RECUAY', 18085],
 [21800, 'SANTA', 474053],
 [21900, 'SIHUAS', 28630],
 [22000, 'YUNGAY', 55769],
 [30100, 'ABANCAY', 120116],
 [30200, 'ANDAHUAYLAS', 150758],
 [30300, 'ANTABAMBA', 11781],
 [30400, 'AYMARAES', 24570],
 [30500, 'COTABAMBAS', 55208],
 [30600, 'CHINCHEROS', 46544],
 [30700, 'GRAU

In [9]:
poblacion =[]
for x in range(len(poblacion_dict['data'])):
    array = poblacion_dict['data'][x][2]
    poblacion.append(array)
    
poblacion

[63188,
 84672,
 26830,
 51344,
 47827,
 33651,
 119294,
 185276,
 6433,
 13950,
 7710,
 24012,
 50007,
 18496,
 57256,
 8017,
 63264,
 33066,
 56557,
 21787,
 7224,
 24371,
 26675,
 18085,
 474053,
 28630,
 55769,
 120116,
 150758,
 11781,
 24570,
 55208,
 46544,
 21759,
 1175765,
 61708,
 43690,
 34743,
 97458,
 16426,
 54851,
 12797,
 317801,
 32482,
 8341,
 97205,
 75277,
 51838,
 29139,
 9909,
 9292,
 19866,
 17063,
 388170,
 83167,
 83916,
 151714,
 29357,
 123948,
 83913,
 203724,
 145770,
 51678,
 47114,
 22638,
 38602,
 1129854,
 511019,
 24000,
 63131,
 71582,
 34754,
 106476,
 70143,
 62059,
 167910,
 26644,
 47579,
 101735,
 70043,
 121265,
 37503,
 53901,
 14588,
 33883,
 18182,
 85995,
 315799,
 53247,
 32427,
 16372,
 52095,
 138275,
 29160,
 50086,
 36987,
 17114,
 18705,
 445752,
 262110,
 78472,
 14832,
 174016,
 595183,
 59138,
 167385,
 88405,
 22757,
 239105,
 91849,
 40041,
 57604,
 1118724,
 123480,
 15982,
 86411,
 30987,
 85091,
 112970,
 85092,
 168670,
 55868

In [10]:
provincia =[]
for x in range(len(poblacion_dict['data'])):
    array = poblacion_dict['data'][x][1]
    provincia.append(array)
    
provincia

['CHACHAPOYAS',
 'BAGUA',
 'BONGARA',
 'CONDORCANQUI',
 'LUYA',
 'RODRIGUEZ DE MENDOZA',
 'UTCUBAMBA',
 'HUARAZ',
 'AIJA',
 'ANTONIO RAYMONDI',
 'ASUNCION',
 'BOLOGNESI',
 'CARHUAZ',
 'CARLOS FERMIN FITZCARRALD',
 'CASMA',
 'CORONGO',
 'HUARI',
 'HUARMEY',
 'HUAYLAS',
 'MARISCAL LUZURIAGA',
 'OCROS',
 'PALLASCA',
 'POMABAMBA',
 'RECUAY',
 'SANTA',
 'SIHUAS',
 'YUNGAY',
 'ABANCAY',
 'ANDAHUAYLAS',
 'ANTABAMBA',
 'AYMARAES',
 'COTABAMBAS',
 'CHINCHEROS',
 'GRAU',
 'AREQUIPA',
 'CAMANA',
 'CARAVELI',
 'CASTILLA',
 'CAYLLOMA',
 'CONDESUYOS',
 'ISLAY',
 'LA UNION',
 'HUAMANGA',
 'CANGALLO',
 'HUANCA SANCOS',
 'HUANTA',
 'LA MAR',
 'LUCANAS',
 'PARINACOCHAS',
 'PAUCAR DEL SARA SARA',
 'SUCRE',
 'VICTOR FAJARDO',
 'VILCAS HUAMAN',
 'CAJAMARCA',
 'CAJABAMBA',
 'CELENDIN',
 'CHOTA',
 'CONTUMAZA',
 'CUTERVO',
 'HUALGAYOC',
 'JAEN',
 'SAN IGNACIO',
 'SAN MARCOS',
 'SAN MIGUEL',
 'SAN PABLO',
 'SANTA CRUZ',
 'CALLAO',
 'CUSCO',
 'ACOMAYO',
 'ANTA',
 'CALCA',
 'CANAS',
 'CANCHIS',
 'CHUMBIVILCAS',


In [11]:
res = {provincia[i]: poblacion[i] for i in range(len(provincia))}
res

{'CHACHAPOYAS': 63188,
 'BAGUA': 84672,
 'BONGARA': 26830,
 'CONDORCANQUI': 51344,
 'LUYA': 47827,
 'RODRIGUEZ DE MENDOZA': 33651,
 'UTCUBAMBA': 119294,
 'HUARAZ': 185276,
 'AIJA': 6433,
 'ANTONIO RAYMONDI': 13950,
 'ASUNCION': 7710,
 'BOLOGNESI': 24012,
 'CARHUAZ': 50007,
 'CARLOS FERMIN FITZCARRALD': 18496,
 'CASMA': 57256,
 'CORONGO': 8017,
 'HUARI': 63264,
 'HUARMEY': 33066,
 'HUAYLAS': 56557,
 'MARISCAL LUZURIAGA': 21787,
 'OCROS': 7224,
 'PALLASCA': 24371,
 'POMABAMBA': 26675,
 'RECUAY': 18085,
 'SANTA': 474053,
 'SIHUAS': 28630,
 'YUNGAY': 55769,
 'ABANCAY': 120116,
 'ANDAHUAYLAS': 150758,
 'ANTABAMBA': 11781,
 'AYMARAES': 24570,
 'COTABAMBAS': 55208,
 'CHINCHEROS': 46544,
 'GRAU': 21759,
 'AREQUIPA': 1175765,
 'CAMANA': 61708,
 'CARAVELI': 43690,
 'CASTILLA': 34743,
 'CAYLLOMA': 97458,
 'CONDESUYOS': 16426,
 'ISLAY': 54851,
 'LA UNION': 12797,
 'HUAMANGA': 317801,
 'CANGALLO': 32482,
 'HUANCA SANCOS': 8341,
 'HUANTA': 97205,
 'LA MAR': 75277,
 'LUCANAS': 51838,
 'PARINACOCHAS':

In [12]:
test['POBLACION'] = test['PROVINCIA'].map(res)

In [13]:
null_columns = test.columns[test.isnull().any()]
test[test["SEXO"].isnull()][null_columns]

,PROVINCIA,DISTRITO,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,POBLACION
1851639,LIMA,LIMA,0.0,NaN,2021-03-13,150101.0,9674755.0


In [14]:
find_provincia = test['PROVINCIA'] == 'LIMA'
find_distrito = test['DISTRITO'] == 'LIMA'
find_fecha = test['FECHA_RESULTADO'] == '2021-03-13'

data_exploratoria = test[find_provincia & find_distrito & find_fecha]
data_exploratoria

,FECHA_CORTE,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,METODODX_y,POBLACION,INDICE
3097,20210817,41740bb3-7baa-4acb-93f6-ed8f16459618,LIMA,LIMA,LIMA,PR,25.0,FEMENINO,2021-03-13,150101.0,932895,9674755.0,8777
29769,20210817,27327fc4-8d01-4ce8-88fc-d010e993e724,LIMA,LIMA,LIMA,PR,29.0,FEMENINO,2021-03-13,150101.0,932895,9674755.0,8777
57253,20210817,6c867a7f-d28e-40a0-89a8-0b22300c05a0,LIMA,LIMA,LIMA,PR,18.0,FEMENINO,2021-03-13,150101.0,932895,9674755.0,8777
57895,20210817,7eb9d4f7-99d1-4cb1-8f2c-54b0d6e6fa91,LIMA,LIMA,LIMA,PR,34.0,MASCULINO,2021-03-13,150101.0,932895,9674755.0,8777
58298,20210817,e41df917-2293-4b90-a067-6e9071c673b9,LIMA,LIMA,LIMA,PR,37.0,MASCULINO,2021-03-13,150101.0,932895,9674755.0,8777
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885870,20210817,79889788-a238-463e-b241-7ce29eab67c8,LIMA,LIMA,LIMA,PCR,0.0,FEMENINO,2021-03-13,150101.0,932895,9674755.0,8777
1910952,20210817,e6aba7ba-b56e-4712-8eca-007302e48669,LIMA,LIMA,LIMA,PCR,70.0,FEMENINO,2021-03-13,150101.0,932895,9674755.0,8777
2073542,20210817,d967fbe5-e7db-493f-8d54-c58f807ef81d,LIMA,LIMA,LIMA,AG,37.0,MASCULINO,2021-03-13,150101.0,932895,9674755.0,8777
2073817,20210817,e84e99cf-43e2-40bd-a816-3f7b20e48a76,LIMA,LIMA,LIMA,AG,47.0,MASCULINO,2021-03-13,150101.0,932895,9674755.0,8777


In [15]:
data_exploratoria.groupby("SEXO").count()

,FECHA_CORTE,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,FECHA_RESULTADO,UBIGEO,METODODX_y,POBLACION,INDICE
SEXO,,,,,,,,,,,,
FEMENINO,88,88,88,88,88,88,88,88,88,88,88,88
MASCULINO,87,87,87,87,87,87,87,87,87,87,87,87


In [16]:
test['SEXO'] = test['SEXO'].fillna('FEMENINO')

In [17]:
null_columns = test.columns[test.isnull().any()]
test[test["PROVINCIA"].isnull()][null_columns]

,PROVINCIA,DISTRITO,EDAD,FECHA_RESULTADO,UBIGEO,POBLACION
8,NaN,NaN,41.0,2020-03-30,NaN,NaN
10,NaN,NaN,24.0,2020-03-30,NaN,NaN
14,NaN,NaN,1.0,2020-03-30,NaN,NaN
17,NaN,NaN,43.0,2020-03-30,NaN,NaN
22,NaN,NaN,62.0,2020-03-30,NaN,NaN
...,...,...,...,...,...,...
2137152,NaN,NaN,62.0,2020-12-30,NaN,NaN
2137193,NaN,NaN,67.0,2021-01-10,NaN,NaN
2137197,NaN,NaN,69.0,2020-12-02,NaN,NaN
2137204,NaN,NaN,66.0,2020-12-20,NaN,NaN


In [18]:
test['FECHA_RESULTADO'] = pd.to_datetime(test['FECHA_RESULTADO'], errors='coerce', dayfirst=True)
test['FECHA_RESULTADO']

0         2020-03-29
1         2020-03-30
2         2020-03-30
3         2020-03-30
4         2020-03-30
             ...    
2137290   2021-01-07
2137291   2020-12-25
2137292   2020-12-10
2137293   2020-12-12
2137294   2020-12-15
Name: FECHA_RESULTADO, Length: 2137295, dtype: datetime64[ns]

In [19]:
test['POBLACION_CIENMIL'] = test['POBLACION']/100000

#test = test.drop(labels="POBLACION_CIENMIL", axis=1)
test

,FECHA_CORTE,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
0,20210817,d4cdd94c-3268-4e80-8296-d343cec109fb,LAMBAYEQUE,FERREÑAFE,FERREÑAFE,PR,35.0,FEMENINO,2020-03-29,140201.0,60386,107241.0,4607,1.07241
1,20210817,586bce7f-3007-4e00-80a3-47fb44fa05ef,LIMA,LIMA,CHORRILLOS,PR,36.0,MASCULINO,2020-03-30,150108.0,932895,9674755.0,8777,96.74755
2,20210817,0520eb2d-b20b-4efb-b391-b155c24e2501,LIMA,LIMA,LIMA,PR,1.0,FEMENINO,2020-03-30,150101.0,932895,9674755.0,8777,96.74755
3,20210817,c59cd628-7031-48be-af21-9dd22d403ff9,LIMA,LIMA,LIMA,PR,65.0,FEMENINO,2020-03-30,150101.0,932895,9674755.0,8777,96.74755
4,20210817,900bbb90-10f2-4c0f-be7e-e8b34b0523cd,LIMA,LIMA,LIMA,PR,32.0,FEMENINO,2020-03-30,150101.0,932895,9674755.0,8777,96.74755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137290,20210817,a84f4445-9608-4851-84d2-59b0b159cf6e,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PCR,64.0,MASCULINO,2021-01-07,150132.0,932895,9674755.0,8777,96.74755
2137291,20210817,92c03f05-13f7-411b-8f8f-07932efdea6a,LIMA,LIMA,LIMA,PCR,62.0,MASCULINO,2020-12-25,150101.0,932895,9674755.0,8777,96.74755
2137292,20210817,a589547e-9ba5-404f-8a47-cae3bda70f13,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PCR,61.0,MASCULINO,2020-12-10,150132.0,932895,9674755.0,8777,96.74755
2137293,20210817,d7ce8e32-ec0d-4f41-a9e1-30b536006556,LIMA,LIMA,SAN MIGUEL,PCR,62.0,MASCULINO,2020-12-12,150136.0,932895,9674755.0,8777,96.74755


In [20]:
salidasxsemanas = test.sort_values(by = 'FECHA_RESULTADO')
start_date = "2021-06-21"
#filtro_extra = "LORETO"

after_start_date = salidasxsemanas["FECHA_RESULTADO"] >= start_date
#also_filtro = salidasxsemanas["PROVINCIA"] == filtro_extra
between_two_dates = after_start_date

filtered_dates = salidasxsemanas.loc[between_two_dates]

filtered_dates

,FECHA_CORTE,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
996250,20210817,76d37e5b-0aed-4a39-9112-73b4c4ce3f6a,LORETO,MAYNAS,ALTO NANAY,AG,44.0,FEMENINO,2021-06-21,160102.0,44645,550551.0,4345,5.50551
1344452,20210817,e90a518a-7b19-4092-8cb1-400636797371,APURIMAC,ANDAHUAYLAS,PAMPACHIRI,AG,39.0,FEMENINO,2021-06-21,30210.0,25720,150758.0,5971,1.50758
1690029,20210817,3f35e54e-bd78-4f88-92f8-47350b35ae7f,PIURA,PIURA,CASTILLA,PCR,48.0,MASCULINO,2021-06-21,200104.0,85300,894847.0,4165,8.94847
1350285,20210817,9fb6c932-c395-4663-b1f4-3b4531df175a,HUANUCO,HUANUCO,AMARILIS,AG,46.0,MASCULINO,2021-06-21,100102.0,35013,315799.0,4605,3.15799
1690006,20210817,8af7fa5f-1c52-43a5-93b1-9da29ce7641a,LIMA,LIMA,SANTIAGO DE SURCO,PCR,53.0,FEMENINO,2021-06-21,150140.0,932895,9674755.0,8777,96.74755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387475,20210817,71e10a34-c8b0-4fc3-bce8-34abfd771545,LIMA,LIMA,JESUS MARIA,AG,27.0,FEMENINO,2021-08-17,150113.0,932895,9674755.0,8777,96.74755
1387478,20210817,ee679db0-5912-4375-8576-fe0ac951d2a8,ANCASH,HUARAZ,HUARAZ,AG,40.0,FEMENINO,2021-08-17,20101.0,75502,185276.0,6395,1.85276
1387480,20210817,13abf99d-5b96-45c3-8356-9133eb078ceb,HUANUCO,HUANUCO,HUANUCO,AG,21.0,MASCULINO,2021-08-17,100101.0,35013,315799.0,4605,3.15799
1387484,20210817,10647139-fb6e-4065-9bfb-9efac7fafc2e,ANCASH,HUARAZ,INDEPENDENCIA,AG,42.0,FEMENINO,2021-08-17,20105.0,75502,185276.0,6395,1.85276


In [21]:
filtered_dates.isnull().sum()

FECHA_CORTE             0
UUID                    0
DEPARTAMENTO            0
PROVINCIA            2479
DISTRITO             2479
METODODX_x              0
EDAD                    2
SEXO                    0
FECHA_RESULTADO         0
UBIGEO               2479
METODODX_y              0
POBLACION            2683
INDICE                  0
POBLACION_CIENMIL    2683
dtype: int64

In [27]:
weekly_sales = filtered_dates.groupby(["SEXO","DEPARTAMENTO","PROVINCIA", "POBLACION", "POBLACION_CIENMIL", "INDICE", pd.Grouper(key="FECHA_RESULTADO",freq="W-MON")]).size()
weekly_sales = weekly_sales.unstack(0).fillna(0)
weekly_sales.sort_values(by=['FECHA_RESULTADO'], inplace=True, ascending=True)
weekly_sales.loc[:,'TOTAL'] = weekly_sales.sum(numeric_only=True, axis=1)
weekly_sales = weekly_sales.reset_index()
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,AMAZONAS,BAGUA,84672.0,0.84672,7273,2021-06-21,13.0,8.0,21.0
1,LIMA,OYON,17137.0,0.17137,8777,2021-06-21,1.0,3.0,4.0
2,ANCASH,YUNGAY,55769.0,0.55769,6395,2021-06-21,1.0,2.0,3.0
3,LIMA,LIMA,9674755.0,96.74755,8777,2021-06-21,360.0,414.0,774.0
4,LIMA,HUAURA,240717.0,2.40717,8777,2021-06-21,19.0,15.0,34.0
...,...,...,...,...,...,...,...,...,...
1677,JUNIN,CHANCHAMAYO,167385.0,1.67385,6108,2021-08-23,2.0,2.0,4.0
1678,SAN MARTIN,SAN MARTIN,218074.0,2.18074,5332,2021-08-23,2.0,1.0,3.0
1679,LORETO,LORETO,71904.0,0.71904,4345,2021-08-23,0.0,1.0,1.0
1680,HUANUCO,PUERTO INCA,36987.0,0.36987,4605,2021-08-23,0.0,1.0,1.0


In [33]:
cero_cases = weekly_sales['TOTAL']==0
data_cero = weekly_sales[cero_cases]
data_cero

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
896,LIMA,CAÑETE,250420.0,2.50420,8777,2021-07-26,0.0,0.0,0.0
1224,LIMA,CAÑETE,250420.0,2.50420,8777,2021-08-02,0.0,0.0,0.0
1355,LIMA,CAÑETE,250420.0,2.50420,8777,2021-08-09,0.0,0.0,0.0
1576,LIMA,CAÑETE,250420.0,2.50420,8777,2021-08-16,0.0,0.0,0.0
1605,UCAYALI,CORONEL PORTILLO,447733.0,4.47733,5422,2021-08-23,0.0,0.0,0.0
1629,ANCASH,SANTA,474053.0,4.74053,6395,2021-08-23,0.0,0.0,0.0
1651,LAMBAYEQUE,LAMBAYEQUE,340835.0,3.40835,4607,2021-08-23,0.0,0.0,0.0


In [28]:
weekly_sales[['FEMENINO','MASCULINO', 'TOTAL']] = weekly_sales[['FEMENINO','MASCULINO', 'TOTAL']].div(weekly_sales['POBLACION_CIENMIL'].values,axis=0)
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,AMAZONAS,BAGUA,84672.0,0.84672,7273,2021-06-21,15.353364,9.448224,24.801587
1,LIMA,OYON,17137.0,0.17137,8777,2021-06-21,5.835327,17.505981,23.341308
2,ANCASH,YUNGAY,55769.0,0.55769,6395,2021-06-21,1.793111,3.586222,5.379333
3,LIMA,LIMA,9674755.0,96.74755,8777,2021-06-21,3.721024,4.279178,8.000203
4,LIMA,HUAURA,240717.0,2.40717,8777,2021-06-21,7.893086,6.231384,14.124470
...,...,...,...,...,...,...,...,...,...
1677,JUNIN,CHANCHAMAYO,167385.0,1.67385,6108,2021-08-23,1.194850,1.194850,2.389700
1678,SAN MARTIN,SAN MARTIN,218074.0,2.18074,5332,2021-08-23,0.917120,0.458560,1.375680
1679,LORETO,LORETO,71904.0,0.71904,4345,2021-08-23,0.000000,1.390743,1.390743
1680,HUANUCO,PUERTO INCA,36987.0,0.36987,4605,2021-08-23,0.000000,2.703653,2.703653


In [29]:
weekly_sales.FEMENINO = weekly_sales.FEMENINO.round()
weekly_sales.MASCULINO = weekly_sales.MASCULINO.round()
weekly_sales.TOTAL = weekly_sales.FEMENINO + weekly_sales.MASCULINO
weekly_sales.FECHA_RESULTADO = weekly_sales.FECHA_RESULTADO.dt.strftime('%Y-%m-%d')
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,AMAZONAS,BAGUA,84672.0,0.84672,7273,2021-06-21,15.0,9.0,24.0
1,LIMA,OYON,17137.0,0.17137,8777,2021-06-21,6.0,18.0,24.0
2,ANCASH,YUNGAY,55769.0,0.55769,6395,2021-06-21,2.0,4.0,6.0
3,LIMA,LIMA,9674755.0,96.74755,8777,2021-06-21,4.0,4.0,8.0
4,LIMA,HUAURA,240717.0,2.40717,8777,2021-06-21,8.0,6.0,14.0
...,...,...,...,...,...,...,...,...,...
1677,JUNIN,CHANCHAMAYO,167385.0,1.67385,6108,2021-08-23,1.0,1.0,2.0
1678,SAN MARTIN,SAN MARTIN,218074.0,2.18074,5332,2021-08-23,1.0,0.0,1.0
1679,LORETO,LORETO,71904.0,0.71904,4345,2021-08-23,0.0,1.0,1.0
1680,HUANUCO,PUERTO INCA,36987.0,0.36987,4605,2021-08-23,0.0,3.0,3.0


In [30]:
weekly_sales.to_csv('dataset_covid_total.csv' , index=False)

In [31]:
weekly_sales.to_json('dataset_covid_total.json', orient="table")